In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from extraction import extract_patches
from reconstruction import perform_voting

from model3d import Multimodel

Using TensorFlow backend.


In [5]:
import os
import nibabel as nib

from medical_data import cdr_info, nwbv_info, diff_info

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

step = (16, 16, 16)
threshold = np.int32(0.30 * np.prod(curr_patch_shape[:]))
seg_train = np.empty((0, 2, ) + curr_patch_shape)
ref_train = np.empty((0, 2, ) + curr_patch_shape)
out_train = np.empty((0, 1, ) + curr_patch_shape)
for i in np.append(range(1, 76), range(80, 100)) :
    print '{} :'.format(i),
    
    for j in range(1, 6) :
        ref_filename = orig_pattern.format(i, j)

        if not os.path.exists(ref_filename) :
            continue
        
        volume_init = nib.load(ref_filename).get_data()

        mask_patches = extract_patches(volume_init != 0, curr_patch_shape, step)

        useful_patches = np.sum(mask_patches, axis=(1, 2, 3)) > threshold

        N = np.sum(useful_patches)

#         volume_init = (volume_init - volume_init.mean()) / volume_init.std()
        volume_init = volume_init / volume_init.max()

        ref_patches = extract_patches(volume_init, curr_patch_shape, step)
        ref_patches = ref_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)

        ref_prob_patches = np.empty((N, 1, ) + curr_patch_shape)
        for a in range(1) :
            prob_filename = prob_pattern.format(i, j, a)
            prob_mask_init = nib.load(prob_filename).get_data()
            prob_patches = extract_patches(prob_mask_init, curr_patch_shape, step)
            prob_patches = prob_patches[useful_patches].reshape((-1, ) + curr_patch_shape)

            ref_prob_patches[:, a] = prob_patches

            del prob_patches
        
        diff = 0
        for k in range(j+1, 6) :
            mov_filename = orig_pattern.format(i, k)

            if not os.path.exists(mov_filename) :
                continue
    
            diff = diff + diff_info['OAS2_00{0:02}_MR{1}'.format(i, k)]
            if diff == 0 :
                continue

            volume_init = nib.load(mov_filename).get_data()
#             volume_init = (volume_init - volume_init.mean()) / volume_init.std()
            volume_init = volume_init / volume_init.max()

            mov_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_patches = mov_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)

            ref_train = np.vstack((np.hstack((ref_patches, ref_prob_patches)), ref_train)) ##
            out_train = np.vstack((mov_patches, out_train)) ##

            mov_prob_patches = np.empty((N, 1, ) + curr_patch_shape)
            for a in range(1) :
                prob_filename = prob_pattern.format(i, k, a)
                prob_mask_init = nib.load(prob_filename).get_data()
                prob_patches = extract_patches(prob_mask_init, curr_patch_shape, step)
                prob_patches = prob_patches[useful_patches].reshape((-1, ) + curr_patch_shape)

                mov_prob_patches[:, a] = prob_patches

                del prob_patches

            seg_train = np.vstack((np.hstack((mov_prob_patches, mov_patches != 0)), seg_train)) ##
            print '{}->{}'.format(j, k),
            del volume_init, mov_patches, mov_prob_patches
        del ref_patches, ref_prob_patches
    print

1 : 1->2
2 : 1->2 1->3 2->3
3 :
4 : 1->2
5 : 1->3 2->3
6 :
7 : 1->3 1->4 3->4
8 : 1->2
9 : 1->2
10 : 1->2
11 :
12 : 1->2 1->3 2->3
13 : 1->3 2->3
14 :
15 :
16 : 1->2
17 : 1->3 1->4 1->5 3->5 4->5
18 : 1->4 3->4
19 :
20 : 1->2 1->3 2->3
21 : 1->2
22 : 1->2
23 : 1->2
24 :
25 :
26 : 1->2
27 : 1->3 1->4 2->3 2->4
28 : 1->2
29 :
30 :
31 : 1->3 2->3
32 :
33 :
34 : 1->4 2->4 3->4
35 :
36 : 1->3 1->4 1->5 3->4 3->5 4->5
37 : 1->2 1->3 1->4 2->3 2->4
38 :
39 : 1->2
40 : 1->3 2->3
41 : 1->3 2->3
42 : 1->2
43 : 1->2
44 : 1->2 1->3
45 : 1->2
46 :
47 :
48 : 1->2 1->3 1->4 1->5 2->3 2->4 2->5 3->4 3->5 4->5
49 :
50 : 1->2
51 : 1->2 1->3 2->3
52 : 1->2
53 : 1->2
54 : 1->2
55 : 1->2
56 : 1->2
57 : 1->3 2->3
58 : 1->3 2->3
59 :
60 : 1->2
61 : 1->2 1->3 2->3
62 : 1->2 1->3 2->3
63 : 1->2
64 : 1->3 2->3
65 :
66 :
67 : 1->2 1->3 1->4 2->3 2->4
68 : 1->2
69 :
70 : 1->2 1->3 1->4 1->5 2->3 2->4 2->5 3->4 3->5
71 : 1->2
72 :
73 : 1->2 1->3 1->4 1->5 2->3 2->4 2->5 3->4 3->5 4->5
74 :
75 :
80 : 1->2 1->3 2->3

In [8]:
curr_patch_shape = (32, 32, 32)
input_modalities = ['T1', 'seg']
output_modalities = ['Gen']
output_weights = {'Gen' : 1.0, 'concat' : 1.0}
latent_dim = 16
channels = [2, 2]
patch_shape = curr_patch_shape
scale = 0.5
a_model = Multimodel(
    input_modalities, output_modalities, output_weights, latent_dim, channels, patch_shape, scale)
a_model.build()

Latent dimensions: 16
Fuse latent representations using max
making output: Tensor("enc_T1_act9_2/add:0", shape=(?, 16, 32, 32, 32), dtype=float32) Tensor("dec_Gen_6/dec_Gen_act5/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_0_dec_Gen
making output: Tensor("enc_seg_act9_2/add:0", shape=(?, 16, 32, 32, 32), dtype=float32) Tensor("dec_Gen_7/dec_Gen_act5/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_1_dec_Gen
making output: Tensor("combined_em_2/Maximum:0", shape=(?, 16, 32, 32, 32), dtype=float32) Tensor("dec_Gen_8/dec_Gen_act5/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_2_dec_Gen
making output: em_concat Tensor("em_concat_2/concat:0", shape=(?, 2, ?), dtype=float32) em_concat_2/concat:0
all outputs:  [u'em_0_dec_Gen_2/add:0', u'em_1_dec_Gen_2/add:0', u'em_2_dec_Gen_2/add:0', u'em_concat_2/concat:0', u'em_fused_2/ExpandDims:0']
output dict:  {'em_concat': <function embedding_distance at 0x7ff70c185f50>, 'em_fused': <function embedding_distance at 0x7ff70c185f50>, 'em_1

In [9]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

patience = 3

stopper = EarlyStopping(patience=patience)
checkpointer = ModelCheckpoint('model3d_scale.h5', save_best_only=True, save_weights_only=True)

N = len(ref_train)
a_model.model.fit(
    [ref_train, seg_train],
    [out_train, out_train, out_train, np.empty((N, 2, 0)), np.empty((N, 1, 0))],
    validation_split=0.3, epochs=40,
    callbacks=[checkpointer, stopper])

Train on 32201 samples, validate on 13801 samples
Epoch 1/40
32201/32201 [==============================] - 1021s 32ms/step - loss: 0.0919 - em_0_dec_Gen_loss: 0.0309 - em_1_dec_Gen_loss: 0.0354 - em_2_dec_Gen_loss: 0.0238 - em_concat_loss: 0.0017 - em_fused_loss: 0.0000e+00 - val_loss: 0.0864 - val_em_0_dec_Gen_loss: 0.0365 - val_em_1_dec_Gen_loss: 0.0272 - val_em_2_dec_Gen_loss: 0.0216 - val_em_concat_loss: 9.8749e-04 - val_em_fused_loss: 0.0000e+00
Epoch 2/40
32201/32201 [==============================] - 1014s 32ms/step - loss: 0.0714 - em_0_dec_Gen_loss: 0.0277 - em_1_dec_Gen_loss: 0.0246 - em_2_dec_Gen_loss: 0.0182 - em_concat_loss: 8.6802e-04 - em_fused_loss: 0.0000e+00 - val_loss: 0.0823 - val_em_0_dec_Gen_loss: 0.0362 - val_em_1_dec_Gen_loss: 0.0248 - val_em_2_dec_Gen_loss: 0.0205 - val_em_concat_loss: 7.5222e-04 - val_em_fused_loss: 0.0000e+00
Epoch 3/40
32201/32201 [==============================] - 1014s 31ms/step - loss: 0.0663 - em_0_dec_Gen_loss: 0.0269 - em_1_dec_Gen_lo

KeyboardInterrupt: 

In [10]:
a_model.model.load_weights('model3d_scale.h5')

In [57]:
def mad(A, B) :
    intersect = np.multiply(A != 0, B != 0)
    max_a = A[intersect].max()
    max_b = B[intersect].max()
    return np.median(np.abs(A[intersect] / max_a - B[intersect] / max_b))

def ssim(A_, B_) :
    A = A_ / A_.max()
    B = B_ / B_.max()
    intersect = np.multiply(A != 0, B != 0)
    ua = A[intersect].mean()
    ub = B[intersect].mean()
    oa = A[intersect].std() ** 2
    ob = B[intersect].std() ** 2
    oab = np.sum(np.multiply(A[intersect] - ua, B[intersect] - ub)) / (np.sum(intersect) - 1)
    k1 = 0.01
    k2 = 0.03
    L = 1
    c1 = (k1 * L) ** 2
    c2 = (k2 * L) ** 2
    num = (2*ua*ub + c1) * (2*oab + c2)
    den = (ua**2 + ub**2 + c1) * (oa + ob + c2)
    return num / den

In [58]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

ref = 1
mov = 3
step = (16, 16, 16)
for i in range(76, 80) :
    ref_filename = orig_pattern.format(i, ref)
    mov_filename = orig_pattern.format(i, mov)
    if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
        continue
    
    print i

    volume_data = nib.load(ref_filename)
    volume_actual = volume_data.get_data()
    volume_actual = volume_actual / volume_actual.max()
    ref_patches = extract_patches(volume_actual, curr_patch_shape, step)
    ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
    
    N = len(ref_patches)

    old_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, ref, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, curr_patch_shape, step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        old_patches[:, k] = prob_patches

        del prob_patches
    
    volume_data = nib.load(mov_filename)
    volume_actual = volume_data.get_data() != 0
    mov_patches = extract_patches(volume_actual, curr_patch_shape, step)
    mov_patches = mov_patches.reshape((-1, 1, ) + curr_patch_shape)
    
    new_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, mov, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, curr_patch_shape, step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        new_patches[:, k] = prob_patches

        del prob_patches
    
    pred = a_model.model.predict([np.hstack((ref_patches, old_patches)), np.hstack((new_patches, mov_patches))], verbose=1)[2]
    pred = pred.reshape((-1, ) + curr_patch_shape)
    
    del new_patches, ref_patches
    
    volume = perform_voting(pred, curr_patch_shape, (256, 256, 128), step)
    
    volume_data = nib.load(mov_filename)
    volume_actual = volume_data.get_data()
    volume = volume * volume_actual.max()
    
    volume = np.multiply(volume_actual != 0, volume)

    nib.save(nib.Nifti1Image(volume, volume_data.affine), 'results/result3d_{}_{}_{}.nii.gz'.format(i, ref, mov))
      
    res = sitk.ReadImage('results/result3d_{}_{}_{}.nii.gz'.format(i, ref, mov))

    caster = sitk.CastImageFilter()
    caster.SetOutputPixelType(res.GetPixelID())

    orig = caster.Execute(sitk.ReadImage(orig_pattern.format(i, ref)))

    matcher = sitk.HistogramMatchingImageFilter()
    matcher.SetNumberOfHistogramLevels(1024)
    matcher.SetNumberOfMatchPoints(15)
    matched = matcher.Execute(res, orig)

    sitk.WriteImage(matched, 'results/result3d_cor_{}_{}_{}.nii.gz'.format(i, ref, mov))
    
    volume = nib.load('results/result3d_cor_{}_{}_{}.nii.gz'.format(i, ref, mov)).get_data()
    ref_vol = np.float64(nib.load(ref_filename).get_data())
    mse_refgen = mad(volume, ref_vol)
    ssim_refgen = ssim(volume, ref_vol)
    mse_movgen = mad(volume, volume_actual)
    ssim_movgen = ssim(volume, np.float64(volume_actual))
    
    print '({}, {}), ({}, {})'.format(mse_refgen, mse_movgen, ssim_refgen, ssim_movgen)

76
1575/1575 [==============================] - 20s 13ms/step
(0.0197167442233, 0.013776384873), (0.979091372243, 0.988337120898)
78
1575/1575 [==============================] - 21s 13ms/step
(0.0276124552513, 0.018205392612), (0.92525002193, 0.975855081903)
79
1575/1575 [==============================] - 20s 13ms/step
(0.106200546878, 0.0396980511872), (0.484370017645, 0.914625512096)
